In [1]:
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from ef_dicts import station_type_map
from ef_dicts_2 import station_name_map
from ef_dict_rer import rer_name_map
from ef_sncf import sncf_dict
import json
from unidecode import unidecode

<h4>Stylesheet</h4>

In [2]:
css_style = """
h1 {
    color: black;
    font-family: 'Segoe UI', 'Gill Sans MT', Calibri, 'Trebuchet MS', sans-serif;
    font-size: 35px !important;
    padding-bottom: 10px;
    padding-top: 10px;
    border-bottom: 5px solid navy;
    border-top: 5px solid navy;
    font-variant: small-caps;
    text-align: center;
    margin-bottom: 25px;
}

h3 {
    color : dimgray;    
}

.all {
/*This class is the default class for <div> so it does not interact with jupyter notebook structure */
text-align: justify;
font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
margin-right: 2px;
}

.titlediv {
    /*This class is the default class for <div> so it does not interact with jupyter notebook structure */
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    border: 5px solid navy;
    padding-top: 30px;
    padding-bottom: 30px;
    padding-left: 5px;
    padding-right: 5px;
    margin-bottom: 10px;
    }

.titlediv_2 {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    text-align: center !important;
    }

.titlep {
    font-size : 50px;
    text-align: center !important;
    font-variant: small-caps;
}   

.titlep_2 {
    font-size : 30px;
    text-align: center !important;
    margin-top: 10px;
}  

.title_sp {
    text-align: center !important;
    font-size: 20px !important;
    font-weight: bold;
}

.signature {
    margin-top: 60px;
    padding-top: 15px;
    border-top: 2px solid black;
    text-align: right !important;
    font-family:'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}

.obj {
    text-align: justify;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    margin-right: 100px;
    border: 2px solid lightcoral;
    padding: 15px;
}

p {
    text-align: justify !important;
}

.intro {
    font-style: italic;
}

.conclusion {
    border: 5px solid navy;
    margin-top: 50px;
    padding: 10px;
}

.conclusion h3 {
    color:black;
    font-variant: small-caps;
}

.small-caps {
    font-variant: small-caps;
}

table {
    margin-right: 10px !important;
    font-size: 14px !important;
    text-align: center !important;
}

.table_1 {
    border: 3px solid black !important;
    width: 100% !important;
}

.table_1 td, .table_1, th {
    border: 1px solid black;
}

.head_tr {
    background-color: dimgray !important;
    color: white;
    border: 3px solid black !important;
}

th, td {
    padding: 8px !important;
    text-align: center !important;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    border: 1.5px solid black; 
}


.recap_table td, .recap_table th {
    height: 50px;
    width: 180px;
}

.recap_table th {
    font-size: 18px;
}

.col_1 {
    text-align: right !important;
    width: 50 !important;
}

.col_group {
    width: 200px;
    background-color: #d4c9df !important;
    text-align: center;
    height: 5px !important;
    font-size: 16 !important;
    font-weight: bold;
}

.columns {
    display: flex;
    flex-wrap: wrap;
}

.column {
    flex: 1;
    padding: 10px;
    text-align: justify;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    margin-right: 50px;
}
"""
    
HTML(f'<style>{css_style}</style>')

In [3]:
display(HTML(f"""
<div class = 'all'>
    <h1>Introduction</h1>
    <h2>Origin of Data</h2>
    <p>
        Data on metro and RER A & B were fetched on <a href = 'https://data.ratp.fr/explore/?sort=modified' target = '_blank'>RATP</a>'s website.
    </p>
    <h3>Included Datasets</h3>
    <ul>
        <li><a href = 'https://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/information/' target = '_blank'>trafic-annuel-entrant-par-station-du-reseau-ferre-2021</a></li>
        <li><a href = 'https://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre-2020/information/' target = '_blank'>trafic-annuel-entrant-par-station-du-reseau-ferre-2020</a></li>
        <li><a href = 'https://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre-2019/information/' target = '_blank'>trafic-annuel-entrant-par-station-du-reseau-ferre-2019</a></li>
        <li><a href = 'https://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre-2018/information/' target = '_blank'>trafic-annuel-entrant-par-station-du-reseau-ferre-2018</a></li>
        <li><a href = 'https://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre-2017/information/' target = '_blank'>trafic-annuel-entrant-par-station-du-reseau-ferre-2017</a></li>
        <li><a href = 'https://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre-2016/information/' target = '_blank'>trafic-annuel-entrant-par-station-du-reseau-ferre-2016</a></li>
        <li><a href = 'https://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre-2015/information/' target = '_blank'>trafic-annuel-entrant-par-station-du-reseau-ferre-2015</a></li>
        <li><a href = 'https://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre-2014/information/' target = '_blank'>trafic-annuel-entrant-par-station-du-reseau-ferre-2014</a></li>
        <li><a href = 'https://data.ratp.fr/explore/dataset/trafic-annuel-entrant-par-station-du-reseau-ferre/information/' target = '_blank'>trafic-annuel-entrant-par-station-du-reseau-ferre</a></li>
</div>
"""))

In [4]:
# Importing RATP data: Metro and RER A/B ridership
ratp_2013 = pd.read_csv('original_datasets/trafic-annuel-entrant-par-station-du-reseau-ferre.csv', sep = ';')
ratp_2014 = pd.read_csv('original_datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2014.csv', sep = ';')
ratp_2015 = pd.read_csv('original_datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2015.csv', sep = ';')
ratp_2016 = pd.read_csv('original_datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2016.csv', sep = ';')
ratp_2017 = pd.read_csv('original_datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2017.csv', sep = ';')
ratp_2018 = pd.read_csv('original_datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2018.csv', sep = ';')
ratp_2019 = pd.read_csv('original_datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2019.csv', sep = ';')
ratp_2020 = pd.read_csv('original_datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2020.csv', sep = ';')
ratp_2021 = pd.read_csv('original_datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021.csv', sep = ';')

ratp_2013.loc[308, 'Station'] = "MAIRIE DE MONTROUGE"
ratp_2016 = ratp_2016.drop(['Column 12', 'Column 13', 'Column 14', 'Column 15'], axis = 1)
ratp_2020.loc[326, 'Station'] = 'SAINT-OUEN'
ratp_2021.loc[93, 'Station'] = 'SAINT-OUEN'

ratp_13_21 = [
    ratp_2013,
    ratp_2014,
    ratp_2015,
    ratp_2016,
    ratp_2017,
    ratp_2018,
    ratp_2019,
    ratp_2020,
    ratp_2021    
]

# Creating a feature that stores the number of lines stopping at each metro station
years = range(2013, 2022)

for i, year in enumerate(years):
    ratp_13_21[i] = ratp_13_21[i].rename(columns={'Rang': f'Rang_{year}', 'Trafic': f'Traffic_{year}', 'Arrondissement pour Paris' : 'Arr'})
    ratp_13_21[i][f'Lignes_{year}'] = ratp_13_21[i].apply(
        lambda row: 5 if pd.notna(row['Correspondance_5'])
        else 4 if pd.notna(row['Correspondance_4'])
        else 3 if pd.notna(row['Correspondance_3'])
        else 2 if pd.notna(row['Correspondance_2'])
        else 1 if pd.notna(row['Correspondance_1'])
        else 0, axis = 1
    )
    
    ratp_13_21[i] = ratp_13_21[i].drop(['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5'], axis = 1)

# Merging RATP dfs as one conaining trafic data from 2013 to 2021

df = ratp_13_21[0]

for i in range(1,9,1):
    df = pd.merge(df, ratp_13_21[i], how = 'outer', on = ['Station', 'Réseau', 'Ville'])
    
df = df.drop(['Arr_x', 'Arr_y'], axis = 1)
df.loc[65, 'Arr'] = 5

df['Traffic_2013_norm'] = (df['Traffic_2013']/df['Lignes_2013']).astype('float')
df['Traffic_2014_norm'] = (df['Traffic_2014']/df['Lignes_2014']).astype('float')
df['Traffic_2015_norm'] = (df['Traffic_2015']/df['Lignes_2015']).astype('float')
df['Traffic_2016_norm'] = (df['Traffic_2016']/df['Lignes_2016']).astype('float')
df['Traffic_2017_norm'] = (df['Traffic_2017']/df['Lignes_2017']).astype('float')
df['Traffic_2018_norm'] = (df['Traffic_2018']/df['Lignes_2018']).astype('float')
df['Traffic_2019_norm'] = (df['Traffic_2019']/df['Lignes_2019']).astype('float')
df['Traffic_2020_norm'] = (df['Traffic_2020']/df['Lignes_2020']).astype('float')
df['Traffic_2021_norm'] = (df['Traffic_2021']/df['Lignes_2021']).astype('float')

df = df.drop(['Lignes_2013', 'Lignes_2014', 'Lignes_2015', 'Lignes_2016', 'Lignes_2017', 'Lignes_2018', 'Lignes_2019', 'Lignes_2020'], axis = 1)

col_trad = {
    "Rang_2013" : "Rank_2013",
    "Rang_2014" : "Rank_2014",
    "Rang_2015" : "Rank_2015",
    "Rang_2016" : "Rank_2016",
    "Rang_2017" : "Rank_2017",
    "Rang_2018" : "Rank_2018",
    "Rang_2019" : "Rank_2019",
    "Rang_2020" : "Rank_2020",
    "Rang_2021" : "Rank_2021",
    "Réseau" : "Network",
    "Ville" : "City",
    "Lignes_2021" : "Lines",
}
df = df.rename(columns = col_trad)

# Feature Creation : Mean trafic, 
df['Mean_traffic'] = df.apply(lambda row: np.mean([row['Traffic_2013'], row['Traffic_2014'], row['Traffic_2015'], row['Traffic_2016'], row['Traffic_2017'], row['Traffic_2018'], row['Traffic_2019'], row['Traffic_2020'], row['Traffic_2021']]), axis = 1)
df['SD_traffic'] = df.apply(lambda row: np.std([row['Traffic_2013'], row['Traffic_2014'], row['Traffic_2015'], row['Traffic_2016'], row['Traffic_2017'], row['Traffic_2018'], row['Traffic_2019'], row['Traffic_2020'], row['Traffic_2021']]), axis = 1)

df['Station_type'] = df.apply(lambda row: "minor" if row["Lines"] == 1
                              else "connexion" if row["Lines"] == 2
                              else "hub" if row["Lines"] > 2
                              else None, axis = 1
                             )

df = df.sort_values(by = 'Mean_traffic', ascending = False)
metro = df[df['Network'] == 'Métro']
rer = df[df['Network'] == 'RER']
metro = metro.reset_index().rename(columns = {"index" : "id"})
metro = metro.reset_index().rename(columns = {"index" : "Rank_mean"})
metro = metro.set_index("id")

rer = rer.reset_index().rename(columns = {"index" : "id"})
rer = rer.reset_index().rename(columns = {"index" : "Rank_mean"})
rer = rer.set_index("id")

df = pd.concat([metro, rer]).sort_values(by = 'id')

df['Station_type'] = df['Station'].map(station_type_map)

gps = pd.read_csv('original_datasets/metro-france.csv', sep = ';')
gps_raw = pd.read_csv('original_datasets/metro-france.csv', sep = ';')

gps['dept'] = gps.apply(lambda row: int(str(row['Commune code Insee'])[:2]), axis = 1)     # Extracting the dept number from Insee code
gps = gps[gps['dept'].isin([75, 91, 92, 93, 94, 95, 77, 78])]                              # Limiting the data to Île-de-France depts
gps = gps[gps['finish'] == 1]                                                              # Dropping data from stations still under construction
gps = gps.drop_duplicates(subset = ['Libelle station'])                                    # In this dataset, there are multiple rows if a stations hosts a connexion
gps['Station'] = gps['Libelle station'].map(station_name_map)

cp = pd.read_csv('original_datasets/postcodes.csv', sep = ';', encoding='latin1')
cp = cp.drop('Ligne_5', axis = 1)
cp = cp.drop_duplicates(subset = ['Code_postal'])
cp = cp.rename(columns = {"Nom_de_la_commune" : "City", "Code_postal": "Postcode"})

gps = gps.rename(columns = {'Commune code Insee' : 'Insee'})
gps['Insee'] = gps.apply(lambda row: str(row['Insee']), axis = 1)
cp = cp.rename(columns = {"#Code_commune_INSEE" : "Insee"})
gps = pd.merge(left = gps, right = cp[["Insee", "Postcode"]], on = "Insee", how = "left")

gps2 = pd.read_csv('original_datasets/gares-idf.csv', sep = ';')
gps2_raw = pd.read_csv('original_datasets/gares-idf.csv', sep = ';')

gps2 = gps2[gps2['res_com'].isin(["RER A", "RER B", "RER C", "RER D", "RER E", "TRAIN H", "TRAIN J", "TRAIN K", "TRAIN L", "TRAIN N",
                      "TRAIN P", "TRAIN R", "TRAIN U"])]
gps2["Latitude"] = gps2.apply(lambda row: json.loads(row['Geo Shape'])["coordinates"][1], axis = 1)
gps2["Longitude"] = gps2.apply(lambda row: json.loads(row['Geo Shape'])["coordinates"][0], axis = 1)
gps2 = gps2[["nom_long", "Latitude", "Longitude", "res_com"]].drop_duplicates(subset = "nom_long")
gps2['Station'] = gps2['nom_long'].map(rer_name_map)

coords = pd.concat([gps[['Station', 'Longitude', 'Latitude']], gps2[['Station', 'Longitude', 'Latitude']]])

df = pd.merge(left = df, right = coords, on = "Station", how = "left")
 
df = df.drop(138)            # Drop Funiculaire station
 
df = df[[
    'Station', 'Network', 'Station_type', 'Longitude', 'Latitude', 'City', 'Arr', 'Lines',
    'Traffic_2013', 'Traffic_2014', 'Traffic_2015', 'Traffic_2016', 'Traffic_2017', 'Traffic_2018', 'Traffic_2019',
    'Traffic_2020', 'Traffic_2021', 'Mean_traffic', 'SD_traffic', 'Traffic_2013_norm','Traffic_2014_norm',
    'Traffic_2015_norm', 'Traffic_2016_norm', 'Traffic_2017_norm', 'Traffic_2018_norm', 'Traffic_2019_norm',
    'Traffic_2020_norm', 'Traffic_2021_norm','Rank_2013', 'Rank_2014', 'Rank_2015',  'Rank_2016', 'Rank_2017',
    'Rank_2018', 'Rank_2019', 'Rank_2020', 'Rank_2021', 'Rank_mean'
]]


trains = pd.read_csv('original_datasets/frequentation-gares.csv', sep = ';')
trains_raw = pd.read_csv('original_datasets/frequentation-gares.csv', sep = ';')

out_idf = [
    "Boran-sur-Oise", "Précy-sur-Oise", "Saint-Leu-d'Esserent", "Creil", "Chantilly - Gouvieux",
    "Orry-la-Ville - Coye", "La Borne Blanche", "Gisors", "Trie-Château", "Chaumont-en-Vexin",
    "Liancourt-Saint-Pierre", "Lavilletertre", "Vernon - Giverny", "Marchezais - Broué", 
    "Dreux", "Malesherbes", "Montargis", "Ferrières - Fontenay", "Dordives", "Château-Thierry", 
    "Chézy-sur-Marne", "Nogent-l'Artaud - Charly", "La Ferté-Milon", "Mareuil-sur-Ourcq",
    "Crépy-en-Valois", "Ormoy-Villers", "Nanteuil-le-Haudouin", "Le Plessis-Belleville"
]

trains['dept'] = trains.apply(lambda row: 99 if row['Nom de la gare'] in out_idf
                              else int(str(row['Code postal'])[:2]) if row['Code postal'] >= 10000
                              else int(str(row['Code postal'])[:1]), axis = 1)
trains = trains[trains['dept'].isin([75, 77, 78, 91, 92, 93, 94, 95, 99])]
trains = trains[~trains["Nom de la gare"].str.contains(r".*T13.*")]
trains = trains[~trains["Nom de la gare"].str.contains(r".*T11.*")]
trains['nom_long'] = trains['Nom de la gare'].map(sncf_dict)

trains = trains.drop(['Total Voyageurs + Non voyageurs 2022', "Total Voyageurs + Non voyageurs 2021", 
                      "Total Voyageurs + Non voyageurs 2020", "Total Voyageurs + Non voyageurs 2019",
                      "Total Voyageurs + Non voyageurs 2018", "Total Voyageurs + Non voyageurs 2017",
                      "Total Voyageurs + Non voyageurs 2016", "Total Voyageurs + Non voyageurs 2015",
                      "Code UIC", "Segmentation DRG"
                     ], axis = 1)

trains2 = pd.merge(left = trains, right = gps2, on = 'nom_long', how = 'left')
trains2 = trains2.drop([131, 154, 205, 208, 213, 223, 237, 262, 278, 327, 358, 287, 241, 242, 375, 382])   # Dropping rows related to tram stations

missing_lat = {
 'Guillerval' : 48.37512,
 'Gazeran' : 48.62589,
 'Longjumeau' : 48.70218,
 'Monnerville' : 48.34855,
 "Paris Bercy Bourgogne - Pays d'Auvergne" : 48.83920,
 'Angerville' : 48.31178,
 'Bréval' : 48.94363,
 'Gravigny Balizy' : 48.68533,
 'Massy TGV' : 48.72739,
 'Chilly-Mazarin' : 48.70067,
 'Petit Vaux' : 48.67651
}

missing_long = {
 'Guillerval' : 2.6057,
 'Gazeran' : 1.77177,
 'Longjumeau' : 2.29415,
 'Monnerville' : 2.03201,
 "Paris Bercy Bourgogne - Pays d'Auvergne" : 2.38294,
 'Angerville' : 2.00346,
 'Bréval' : 1.15180,
 'Gravigny Balizy' : 2.31742,
 'Massy TGV' : 2.26338,
 'Chilly-Mazarin' : 2.30818,
 'Petit Vaux' : 2.33272
}

trains2['Latitude'] = trains2.apply(lambda row: missing_lat[row['Nom de la gare']] if row['Nom de la gare'] in missing_lat.keys()
                                    else row['Latitude'], axis = 1)
trains2['Longitude'] = trains2.apply(lambda row: missing_long[row['Nom de la gare']] if row['Nom de la gare'] in missing_long.keys()
                                     else row['Longitude'], axis = 1)
trains2 = trains2.drop("res_com", axis = 1)
trains2 = trains2.rename(columns = {"Nom de la gare" : "Station_name", "Code postal" : "Postcode",
                                   "Total Voyageurs 2022" : "Traffic_2022", "Total Voyageurs 2021" : "Traffic_2021",
                                   "Total Voyageurs 2020" : "Traffic_2020", "Total Voyageurs 2019" : "Traffic_2019",
                                   "Total Voyageurs 2018" : "Traffic_2018", "Total Voyageurs 2017" : "Traffic_2017",
                                   "Total Voyageurs 2016" : "Traffic_2016", "Total Voyageurs 2015" : "Traffic_2015"
                                  })
trains2['Network'] = "RER"


trains2 = pd.merge(left = trains2, right = cp[['City', 'Postcode']], on = "Postcode", how = "left")
df = pd.concat([df, trains2])



df['Station_name'] = df.apply(lambda row: unidecode(str(row['Station']).lower()) if pd.isna(row['Station_name']) else unidecode(str(row['Station_name']).lower()), axis = 1)
#df = df.drop_duplicates(subset = "Station_name")
df = df.drop(['Station', 'nom_long'], axis = 1)

df = df[['Station_name', 'Network', 'Station_type', 'Longitude', 'Latitude', 'City', 'Arr',
       'Lines', 'Traffic_2013', 'Traffic_2014', 'Traffic_2015', 'Traffic_2016',
       'Traffic_2017', 'Traffic_2018', 'Traffic_2019', 'Traffic_2020',
       'Traffic_2021', 'Mean_traffic', 'SD_traffic', 'Traffic_2013_norm',
       'Traffic_2014_norm', 'Traffic_2015_norm', 'Traffic_2016_norm',
       'Traffic_2017_norm', 'Traffic_2018_norm', 'Traffic_2019_norm',
       'Traffic_2020_norm', 'Traffic_2021_norm', 'Rank_2013', 'Rank_2014',
       'Rank_2015', 'Rank_2016', 'Rank_2017', 'Rank_2018', 'Rank_2019',
       'Rank_2020', 'Rank_2021', 'Rank_mean', 'Postcode',
       'Traffic_2022', 'dept']]

df = df.drop_duplicates()

df = df.reset_index(drop = True)

df.to_csv('even_flow_etl_database.csv')
display(HTML(f"""
<div class = 'all'>
    <h2></h2>
    <p>
        Ridership data was gathered from 2013 to 2021. During this time period:
    </p>
    <ul>
        <li>March 2013: Metro station Mairie de Montrouge (Line 4) opened</li>
        <li>2020: Covid-19 outburst, with several lockdown period in France</li>
        <li>2020-2021: Metro line 14 was extended, connecting to Porte de Clichy and Mairie de Saint-Ouen metro stations,
        as well as Pont Cardinet and Saint-Ouen. The latter two are not present in prior datasets as they were stations for
        Transilien line L and RER line C, not covered by these datasets.</li>
        <li>No station was closed during this time period.</li>
    </ul>
    <h2>Data Processing</h2>
    <ul>
        <li>In the 2013 dataset, Mairie de Montrouge station was labeled <code>"MAIRIE DE MONTROUGE**"</code>
        , probably because the station opened in march 2013. "**" was deleted to match the name of this station 
        in other datasets.</li>
        <li>Four empty columns were found in 2016 dataset, and were dropped.</li>
        <li>In 2020 and 2021 datasets, Saint-Ouen station was labeled <code>"CLICHY SAINT-OUEN"</code> as it was the expected
        commercial name, but was eventually named Saint-Ouen to match the existing station on RER C line. 
        Accordingly, station was renamed <code>"SAINT-OUEN"</code> in the present dataset.</li>
        <li>The arrondissement was missing for the station <code>"SAINT-MICHEL NOTRE-DAME"</code> and was manually added.</li>
    </ul>
    <h2>Feature Engineering</h2>
    <ul>
        <li>Features named <code>Correspondance_X</code> were dropped and replaced by a unique feature <code>Line</code>: 
        the number of lines connecting in the station.</li>
        <li>Creation of <code>Mean_trafic</code> and <code>SD_Trafic</code> to calculate the mean 
        and SD of trafic over the covered period.</li>
        <li>Creation of <code>Trafic_norm_X</code>, X being the related year. This feature is the yearly trafic
        divided by the number of lines in the station.</li>
        <li>Creation of <code>Station_type</code> that separated stations into groups, as detailed in the table below.</li>
        <li>Creation of <code>Rank_mean</code>, the rank based on <code>Mean_trafic</code>.</li>
        <li> Creation of <code>Latitude</code> and <code>Longitude</code> to store gps coordinates of all stations.
            <ul>
                <li>Metro station longitude and latitude where obtained from another dataset named <i>Lignes et stations de métro en France</i>
                    from the <a href = "https://www.data.gouv.fr/fr/datasets/lignes-et-stations-de-metro-en-france/" target = '_blank'>french government website</a>.</li>
                <li>RER station longitude and latitude were obtained from another dataset named <i>emplacement-des-gares-idf</i> from <a href = 'https://data.iledefrance-mobilites.fr/explore/dataset/emplacement-des-gares-idf/information/' target = '_blank'>Île de France Mobilités website</a>
                for lines administered by RATP and from a dataset names <i>frequentation-gares</i> from <a href = 'https://ressources.data.sncf.com/explore/dataset/frequentation-gares/information/?disjunctive.nom_gare&disjunctive.code_postal&sort=nom_gare' target = '_blank'>SNCF website</a>
                for lines administered by SNCF.</li>
            </ul>
        </li>
    </ul>
</div>
    </p>
</div>

<div class = 'all'>
    <table style = 'border : 1px solid black'>
        <tr style = 'border: 1px solid black'>
            <th>Station type</th>
            <th style = 'width: 50%'>Definition</th>
            <th>Example</th>
        </tr>
        <tr style = 'border: 1px solid black'>
            <td><strong>Train station</strong></td>
            <td>This group includes all national service train stations, including TGV stations outside of Paris
            such as Marne-la-Vallée - Chessy. Antony station is included as it connects to Orly airport via a dedicated
            train (Orlyval).</td>
            <td>Gare du Nord</td>
        </tr>
        <tr style = 'border: 1px solid black'>
            <td><strong>Metro-RER</strong></td>
            <td>Any station that connects metro and RER.</td>
            <td>Nation</td>
        </tr>
        <tr style = 'border: 1px solid black'>
            <td><strong>RER connexion</strong></td>
            <td>Any station that connects two or more RER lines, but is not a train station.</td>
            <td>Saint-Michel - Notre-Dame</td>
        </tr>
        <tr style = 'border: 1px solid black'>
            <td><strong>Metro hub</strong></td>
            <td>Any metro station that connects more than 2 metro lines, and do not connect to RER.</td>
            <td>La Motte-Picquet - Grenelle</td>
        </tr>
        <tr style = 'border: 1px solid black'>
            <td><strong>Metro connexion</strong></td>
            <td>Any metro station that connects 2 metro lines together</td>
            <td>Place de Clichy</td>
        </tr>
        <tr style = 'border: 1px solid black'>
            <td><strong>Tram connexion</strong></td>
            <td>Metro or RER station where a tram connexion is available (and no other metro or RER connexion)</td>
            <td>Porte de Pantin</td>
        </tr>
        <tr style = 'border: 1px solid black'>
            <td><strong>Local station</strong></td>
            <td>Any metro or RER station that do no connect to any other line.</td>
            <td>Créteil l'Échat</td>
        </tr>
    </table>
</div>
<br>
<div class = 'all' style = 'font-style:italic; font-size: 12px'>
    <p>
    <strong>Semantic note</strong> -  "RER" refers to any suburban train line:
    </p>
    <ul>
        <li>RER line A, B, C, D and E</li>            
        <li>Transilien line H, J, K, L, N, P, R, U</li>
    </ul>
    <p>
        Conversely, "train" refers to other national train lines, usually classified as "TGV" (high speed train), 
        "Intercité" (regular speed train) and "TER" (regional trains). Such trains will connect to a very limited number
        of stations from this dataset, classified as "Train station"
    </p>
</div>
"""))

display(df.head(10))

Station type,Definition,Example
Train station,"This group includes all national service train stations, including TGV stations outside of Paris such as Marne-la-Vallée - Chessy. Antony station is included as it connects to Orly airport via a dedicated train (Orlyval).",Gare du Nord
Metro-RER,Any station that connects metro and RER.,Nation
RER connexion,"Any station that connects two or more RER lines, but is not a train station.",Saint-Michel - Notre-Dame
Metro hub,"Any metro station that connects more than 2 metro lines, and do not connect to RER.",La Motte-Picquet - Grenelle
Metro connexion,Any metro station that connects 2 metro lines together,Place de Clichy
Tram connexion,Metro or RER station where a tram connexion is available (and no other metro or RER connexion),Porte de Pantin
Local station,Any metro or RER station that do no connect to any other line.,Créteil l'Échat


,Station_name,Network,Station_type,Longitude,Latitude,City,Arr,Lines,Traffic_2013,Traffic_2014,Traffic_2015,Traffic_2016,Traffic_2017,Traffic_2018,Traffic_2019,Traffic_2020,Traffic_2021,Mean_traffic,SD_traffic,Traffic_2013_norm,Traffic_2014_norm,Traffic_2015_norm,Traffic_2016_norm,Traffic_2017_norm,Traffic_2018_norm,Traffic_2019_norm,Traffic_2020_norm,Traffic_2021_norm,Rank_2013,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_mean,Postcode,Traffic_2022,dept
0,opera,Métro,Metro hub,2.331047,48.871437,Paris,9.0,3.0,12269711.0,12288027.0,11717705.0,10981914.0,11022100.0,11175606.0,10501357.0,3519216,5193831,9.852163e+06,3.016299e+06,4.089904e+06,4096009.0,3.905902e+06,3660638.00,3.674033e+06,3725202.00,3.500452e+06,1173072.00,1731277.00,12.0,13.0,13.0,15.0,16.0,15.0,15.0,41.0,32.0,14.0,NaN,NaN,NaN
1,hotel de ville,Métro,Metro connexion,2.352068,48.857352,Paris,4.0,2.0,11953352.0,12621901.0,12201851.0,11764439.0,11777638.0,11467832.0,12307363.0,5673100,7251729,1.077991e+07,2.359155e+06,5.976676e+06,6310950.5,6.100926e+06,5882219.50,5.888819e+06,5733916.00,6.153682e+06,2836550.00,3625864.50,13.0,11.0,12.0,12.0,12.0,14.0,11.0,13.0,13.0,11.0,NaN,NaN,NaN
2,nation,Métro,Metro-RER,2.395844,48.848084,Paris,12.0,4.0,9541406.0,9333662.0,8939092.0,8792715.0,8718463.0,8638165.0,8834660.0,4860619,6050797,8.189953e+06,1.513471e+06,2.385352e+06,2333415.5,2.234773e+06,2198178.75,2.179616e+06,2159541.25,2.208665e+06,1215154.75,1512699.25,19.0,21.0,24.0,24.0,25.0,25.0,22.0,16.0,20.0,21.0,NaN,NaN,NaN
3,saint-michel,Métro,Metro-RER,2.343992,48.853594,Paris,5.0,1.0,8797595.0,8695006.0,8072295.0,7562865.0,7469900.0,6713126.0,6286034.0,2481981,3747385,6.647354e+06,2.061732e+06,8.797595e+06,8695006.0,8.072295e+06,7562865.00,7.469900e+06,6713126.00,6.286034e+06,2481981.00,3747385.00,25.0,26.0,29.0,32.0,34.0,48.0,55.0,85.0,65.0,35.0,NaN,NaN,NaN
4,palais-royal,Métro,Metro connexion,2.336454,48.862222,Paris,1.0,2.0,9385220.0,7448640.0,10060631.0,9621986.0,10024410.0,9648312.0,9592920.0,3678654,4822599,8.253708e+06,2.276523e+06,4.692610e+06,3724320.0,5.030316e+06,4810993.00,5.012205e+06,4824156.00,4.796460e+06,1839327.00,2411299.50,20.0,32.0,18.0,20.0,20.0,19.0,18.0,35.0,40.0,20.0,NaN,NaN,NaN
5,crimee,Métro,Local station,2.376936,48.890886,Paris,19.0,1.0,6945245.0,6800500.0,6613387.0,6175537.0,5999504.0,5921134.0,5431969.0,2688758,3543952,5.568887e+06,1.398173e+06,6.945245e+06,6800500.0,6.613387e+06,6175537.00,5.999504e+06,5921134.00,5.431969e+06,2688758.00,3543952.00,42.0,46.0,48.0,50.0,61.0,68.0,74.0,65.0,75.0,54.0,NaN,NaN,NaN
6,grands boulevards,Métro,Local station,2.343207,48.871505,Paris,9.0,2.0,7092775.0,7345811.0,6921912.0,6889717.0,7141016.0,7416725.0,6807424.0,2608284,3737316,6.217887e+06,1.660122e+06,3.546388e+06,3672905.5,3.460956e+06,3444858.50,3.570508e+06,3708362.50,3.403712e+06,1304142.00,1868658.00,39.0,35.0,40.0,44.0,41.0,37.0,44.0,78.0,66.0,44.0,NaN,NaN,NaN
7,pont de neuilly,Métro,Local station,2.258523,48.885499,Neuilly Sur Seine,NaN,1.0,6902931.0,6823045.0,6786803.0,6815851.0,7061761.0,7127658.0,6902027.0,3678074,4809503,6.323073e+06,1.147906e+06,6.902931e+06,6823045.0,6.786803e+06,6815851.00,7.061761e+06,7127658.00,6.902027e+06,3678074.00,4809503.00,43.0,45.0,42.0,46.0,44.0,42.0,41.0,36.0,41.0,41.0,NaN,NaN,NaN
8,mairie de clichy,Métro,Local station,2.305726,48.903449,Clichy,NaN,1.0,6987958.0,7129195.0,7138355.0,7089241.0,7107063.0,7105379.0,6342058.0,3316323,4043071,6.250960e+06,1.404863e+06,6.987958e+06,7129195.0,7.138355e+06,7089241.00,7.107063e+06,7105379.00,6.342058e+06,3316323.00,4043071.00,41.0,36.0,35.0,39.0,42.0,43.0,51.0,47.0,53.0,43.0,NaN,NaN,NaN
9,sevres-babylone,Métro,Metro connexion,2.326860,48.851565,Paris,7.0,2.0,5540265.0,5387363.0,5237966.0,5178469.0,5321593.0,5430593.0,5037509.0,2441636,3392504,4.774211e+06,1.026921e+06,2.770132e+06,2693681.5,2.618983e+06,2589234.50,2.660796e+06,2715296.50,2.518754e+06,1220818.00,1696252.00,67.0,71.0,75.0,78.0,73.0,74.0,80.0,90.0,88.0,75.0,NaN,NaN,Na

In [ ]:
#https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/#/resources
#pour les codes postals

In [5]:
#Postcode : ajouter a métro/rer avant merge
#uniformiser les stations names

In [13]:
df.loc[438, 'Station_name'] = 'porte de clichy rer'
df.loc[446, 'Station_name'] = 'saint-ouen rer'
df.loc[578, 'Station_name'] = 'malesherbes rer'
df.loc[594, 'Station_name'] = 'pont cardinet rer'
df.loc[603, 'Station_name'] = 'saint-fargeau rer'
df.loc[717, 'Station_name'] = 'invalides rer'

In [14]:
df[df['Station_name'].duplicated()] 0664099692

,Station_name,Network,Station_type,Longitude,Latitude,City,Arr,Lines,Traffic_2013,Traffic_2014,Traffic_2015,Traffic_2016,Traffic_2017,Traffic_2018,Traffic_2019,Traffic_2020,Traffic_2021,Mean_traffic,SD_traffic,Traffic_2013_norm,Traffic_2014_norm,Traffic_2015_norm,Traffic_2016_norm,Traffic_2017_norm,Traffic_2018_norm,Traffic_2019_norm,Traffic_2020_norm,Traffic_2021_norm,Rank_2013,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_mean,Postcode,Traffic_2022,dept
494,la defense,RER,NaN,2.237018,48.892187,PUTEAUX,NaN,NaN,NaN,NaN,28392626.0,28638151.0,29470800.0,29752335.0,30264172.0,6688269,24737930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92800.0,35649320.0,92.0
679,val de fontenay,RER,NaN,2.201823,49.091073,FONTENAY SOUS BOIS,NaN,NaN,NaN,NaN,17396627.0,17683219.0,17900299.0,17739617.0,17693555.0,17692029,22148961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94120.0,16450965.0,94.0


In [24]:
df[df['Station_name'].str.contains(r'bond')]

,Station_name,Network,Station_type,Longitude,Latitude,City,Arr,Lines,Traffic_2013,Traffic_2014,Traffic_2015,Traffic_2016,Traffic_2017,Traffic_2018,Traffic_2019,Traffic_2020,Traffic_2021,Mean_traffic,SD_traffic,Traffic_2013_norm,Traffic_2014_norm,Traffic_2015_norm,Traffic_2016_norm,Traffic_2017_norm,Traffic_2018_norm,Traffic_2019_norm,Traffic_2020_norm,Traffic_2021_norm,Rank_2013,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_mean,Postcode,Traffic_2022,dept
542,bondy,RER,NaN,2.480206,48.893952,BONDY,NaN,NaN,NaN,NaN,12955985.0,13017323.0,13024968.0,12799616.0,12574385.0,7412905,11082849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93140.0,14605023.0,93.0


In [ ]:
#FuSiOn !

In [ ]:
Nanterre Université
Nanterre Préfecture
La défense
Chatelet les Halles
Paris Gare de Lyon
Val de Fontenay

Massy Palaiseau
Massy Verrieres
St Michel Notre Dame
Gare du Nord

Saint Ouen
Porte de Clichy
Pereire Levallois
Neuilly Porte Maillot
Boulainvilliers
Champs de Mars Tour Eiffel
Invalides
Gare d\'Austerlitz
BFM

Paris Saint Lazare

Pont Cardinet

Paris Est

